In [1]:

import boto3
import os
from dotenv import load_dotenv
from botocore.exceptions import NoCredentialsError, ClientError
import sys
sys_path = r'../src'
sys.path.append(sys_path)

from s3_manager import S3_Downloader
from config_loader import read_yaml_file
import pandas as pd




In [2]:
# Load environment variables from .env file
load_dotenv(r'../config/config.env')

config = read_yaml_file(r'../config/default_config.yaml')

In [3]:
# Establishing connection with AWS
client_obj = S3_Downloader.establish_connection_s3()        



✅ S3 client created successfully...!!


In [4]:
# Preprocessing Bucket_Prefix Pair: yaml file
bucket_prefix_pair = S3_Downloader.preprocess_bucket_prefix(
    s3_client=client_obj, 
    bucket_prefix_map=config['s3']
)
print('\Bucket Prefix Pairs: ')
[print('\t\t- ', bucket_prefix) for bucket_prefix in bucket_prefix_pair]

	- Bucket Prefix Pairs: 
		-  ('prod-completed', '2025-05-30/')
		-  ('prod-completed', '2025-03-01/')


[None, None]

In [5]:
# Creating Download Path
os.makedirs(config['sync']['loc_download'], exist_ok=True)

# Creating Obj list to be downloaded
s3_obj_list = S3_Downloader.list_s3_objects(
    client=client_obj, 
    bucket_prefixes=bucket_prefix_pair
)

In [6]:
# Applying filters on S3 Obj List
s3_obj_list = S3_Downloader.filter_obj_list(obj_list=s3_obj_list, 
                                              filter_criteria=config['filters'], 
                                              sync_mode=config['sync']['mode'], 
                                             loc_downloads=config['sync']['loc_download'])

In [ ]:
# Execute Download 
S3_Downloader.execute_download(client_obj, s3_obj_list)

In [ ]:
# Save Parquet file
file_utils.save_sync_state(df = s3_obj_list, output_dir = download_path)